In [152]:
import pandas as pd
import yaml

In [153]:
with open('../input/dtypes.yaml', 'r') as yamlfile:
    dtypes = yaml.load(yamlfile)

In [154]:
read_csv_opts = {'sep': '|',
                 'quotechar': '"',
                 'compression': 'gzip',
                 'encoding': 'utf-8',
                 'dtype': dtypes,
                 'parse_dates': ['MissionDate'],
                 'infer_datetime_format': True}

In [155]:
fy11 = '../input/ARTS_Passenger_Data_FY11.csv.gz'
fy12 = '../input/ARTS_Passenger_Data_FY12.csv.gz'
fy13 = '../input/ARTS_Passenger_Data_FY13.csv.gz'
fy14 = '../input/ARTS_Passenger_Data_FY14.csv.gz'
fy15 = '../input/ARTS_Passenger_Data_FY15.csv.gz'
fy16 = '../input/ARTS_Passenger_Data_FY16.csv.gz'
fy17 = '../input/ARTS_Passenger_Data_FY17.csv.gz'
fy18 = '../input/ARTS_Passenger_Data_FY18.csv.gz'
fy19 = '../input/ARTS_Passenger_Data_FY19.csv.gz'

In [156]:
arts_fy11 = pd.read_csv(fy11, **read_csv_opts)
arts_fy12 = pd.read_csv(fy12, **read_csv_opts)
arts_fy13 = pd.read_csv(fy13, **read_csv_opts)
arts_fy14 = pd.read_csv(fy14, **read_csv_opts)
arts_fy15 = pd.read_csv(fy15, **read_csv_opts)
arts_fy16 = pd.read_csv(fy16, **read_csv_opts)
arts_fy17 = pd.read_csv(fy17, **read_csv_opts)
arts_fy18 = pd.read_csv(fy18, **read_csv_opts)
arts_fy19 = pd.read_csv(fy19, **read_csv_opts)

In [157]:
files = [arts_fy11,
         arts_fy12,
         arts_fy13,
         arts_fy14,
         arts_fy15,
         arts_fy16,
         arts_fy17,
         arts_fy18,
         arts_fy19]

In [158]:
df = pd.concat(files, sort=False)

In [159]:
duplicated = df[df.duplicated('AlienMasterID')]

In [160]:
bad_puloc = list(duplicated['PULOC'].unique())

In [161]:
bad_droploc = list(duplicated['DropLoc'].unique())

In [162]:
bad_airports = pd.read_csv('../hand/bad_airports.csv')

In [163]:
df.columns

Index(['Status', 'Sex', 'Convictions', 'GangMember', 'ClassLvl', 'Age',
       'MissionDate', 'MissionNumber', 'PULOC', 'DropLoc', 'StrikeFromList',
       'ReasonStruck', 'R-T', 'Code', 'CountryOfCitizenship', 'Juvenile',
       'MissionWeek', 'MissionQuarter', 'MissionYear', 'MissionMonth',
       'Criminality', 'FamilyUnitFlag', 'UnaccompaniedFlag', 'AlienMasterID',
       'MissionID', 'air_AirportID', 'air_AirportName', 'air_City',
       'st_StateID', 'st_StateAbbr', 'AOR_AORID', 'AOR_AOR', 'AOR_AORName',
       'air_Country', 'air2_AirportID', 'air2_AirportName', 'air2_City',
       'st2_StateID', 'st2_StateAbbr', 'aor2_AORID', 'aor2_AOR',
       'aor2_AORName', 'air2_Country'],
      dtype='object')

In [164]:
df['PULOC'] = df['PULOC'].str.upper()
df['DropLoc'] = df['DropLoc'].str.upper()

In [165]:
metadata = ['PULOC', 'DropLoc','air_AirportID', 'air_AirportName', 'air_City',
            'st_StateID', 'st_StateAbbr', 'AOR_AORID', 'AOR_AOR', 'AOR_AORName',
            'air_Country', 'air2_AirportID', 'air2_AirportName', 'air2_City',
            'st2_StateID', 'st2_StateAbbr', 'aor2_AORID', 'aor2_AOR',
            'aor2_AORName', 'air2_Country']

In [166]:
pu_metadata = ['PULOC', 'air_AirportID', 'air_AirportName', 'air_City',
            'st_StateID', 'st_StateAbbr', 'AOR_AORID', 'AOR_AOR', 'AOR_AORName',
            'air_Country']

In [167]:
do_metadata = ['DropLoc','air2_AirportID', 'air2_AirportName', 'air2_City',
            'st2_StateID', 'st2_StateAbbr', 'aor2_AORID', 'aor2_AOR',
            'aor2_AORName', 'air2_Country']

In [168]:
standard_metadata = ['AirportCode', 'AirportID', 'AirportName', 'City',
            'StateID', 'StateAbbr', 'AORID', 'AOR',
            'AORName', 'Country']

In [169]:
dup_meta = df[df.duplicated('AlienMasterID', keep=False)][metadata].drop_duplicates()

In [170]:
pu_dup_meta = dup_meta[pu_metadata]
do_dup_meta = dup_meta[do_metadata]

In [171]:
pu_dup_meta.columns = standard_metadata
do_dup_meta.columns = standard_metadata

In [172]:
dup_meta = pd.concat([pu_dup_meta, do_dup_meta]).drop_duplicates()

In [173]:
dup_meta = dup_meta.sort_values(by='AirportCode')

In [174]:
dup_meta[dup_meta.AirportCode.isin(bad_airports.airport_code)]

,AirportCode,AirportID,AirportName,City,StateID,StateAbbr,AORID,AOR,AORName,Country
47132,HAAB,29.0,Bole International Airport,Addis Ababa,NaN,NaN,NaN,NaN,NaN,Ethiopia
47133,HAAB,244.0,Addis Ababa Bole International Airport,Bole,NaN,NaN,NaN,NaN,NaN,Ethiopia
86918,KMHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192703,KNYL,91.0,Yuma MCAS/Yuma International Airport,Yuma Naval,4.0,AZ,17.0,PHO,Phoenix,USA
192704,KNYL,90.0,Yuma MCAS/Yuma International Airport,Yuma,3.0,AR,17.0,PHO,Phoenix,USA
105634,ORBI,289.0,Baghdad International Airport,Baghdad,NaN,NaN,NaN,NaN,NaN,Iraq
105635,ORBI,165.0,Baghdad International Airport,Baghdad,NaN,NaN,NaN,NaN,NaN,Iraq


In [175]:
len(df.drop_duplicates('AlienMasterID'))

1733555

In [176]:
df = df[~df['air_AirportID'].isin([29.0, 90.0, 165.0])]
df = df[~df['air2_AirportID'].isin([29.0, 90.0, 165.0])]

In [177]:
len(df)

1733555